In [ ]:
import os
import datetime
import textwrap
import time
import yaml
from pathlib import Path
from huggingface_hub import login

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{int(minutes)} minutes and {int(seconds)} seconds"

# INPUT/OUTPUT, MODEL SIZE Variables
INPUT_FILENAME = "testv2.mp4"
OUTPUT_FILENAME = "output.vtt"
MODEL_SIZE = "small"
MAX_LINE_LENGTH = 30

# Set the name of the configuration file path
CONFIG_FILE_PATH = "/home/idies/workspace/Storage/tsande16/persistent/archive/archive_config.yaml"

start_time = time.time()
print(f"Starting notebook transcription and diarization at {datetime.datetime.now().strftime('%H:%M:%S')}")

with open(CONFIG_FILE_PATH, 'r') as file:
    config = yaml.safe_load(file.read())

# Set config variables
script_path = config['storage']['temp'] + "/transcribe_diarize.py"
input_file = config['storage']['content'] + "/" + INPUT_FILENAME
output_file = config['storage']['output'] + "/" + OUTPUT_FILENAME
cache_dir = Path(config['storage']['hf_cache'])
cache_dir.mkdir(parents=True, exist_ok=True)

# Set the cache dir for HF. Set hugging face token that is stored in the configuration file.
os.environ['HF_HOME'] = str(cache_dir)
print(f"Hugging Face cache directory set to: {cache_dir}")
os.environ['HF_TOKEN'] = config['tokens']['hugging_face']
hf_token = config['tokens']['hugging_face']

# Test that the login worked.
!hf auth whoami

# Get the most recent transcribe_diarize script
!wget https://raw.githubusercontent.com/jhu-sheridan-libraries/whisper_testing/main/transcribe_diarize.py -O {script_path}

if os.path.exists(config['storage']['temp']+"/transcribe_diarize.py"):
  print("Success: The transcribe_diarize.py file downloaded.")
else:
  print("Error: The transcribe_diarize.py file didn't download.")

!python {script_path} {input_file} \
    --whisper_cache_dir {cache_dir} \
    --output {output_file} \
    --model {MODEL_SIZE} \
    --line-length {MAX_LINE_LENGTH}

total_time = time.time() - start_time
print(f"\nTotal notebook runtime: {format_time(total_time)}")

# Download results
if os.path.exists(config['storage']['output']+"/output.vtt"):
  print("VTT file was successfully generated and stored here: ", config['storage']['output']+"/output.vtt")
else:
  print("Error: The VTT file was not generated.")

In [ ]:
# View torch audio and other dependencies versions
!pip show torchaudio
!pip show pyannote.audio
!pip show speechbrain